In [2]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas as pd
import torch
from torch import nn, optim
import numpy as np

from tqdm import tqdm
from torchvision.utils import save_image, make_grid
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim


/data1/dataiku/DATA_DESIGNER/code-envs/python/offus/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [5]:
# =========================
# Masked Variational Autoencoder
# =========================

class MaskedVAE(nn.Module):
    """
    Masked Variational Autoencoder for incomplete data.
    Encoder input: [x ⊙ m , m]
    Latent output: mu (used as latent feature u_i)
    """

    def __init__(self, input_dim, hidden_dim=128, latent_dim=8):
        super().__init__()

        # Encoder: input_dim * 2 because of concatenation with mask
        self.encoder = nn.Sequential(
            nn.Linear(input_dim * 2, hidden_dim),
            nn.LeakyReLU(0.1),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.1),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.1)
        )

        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.LeakyReLU(0.1),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.1),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.1),
            nn.Linear(hidden_dim, input_dim)
        )

    def encode(self, x, mask):
        x_masked = x * mask
        enc_input = torch.cat([x_masked, mask], dim=1)
        h = self.encoder(enc_input)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x, mask):
        mu, logvar = self.encode(x, mask)
        z = self.reparameterize(mu, logvar)
        recon = self.decode(z)
        return recon, mu, logvar


# =========================
# Masked VAE Loss
# =========================

def masked_vae_loss(
    recon_x,
    x,
    mask,
    mu,
    logvar,
    recon_weight=1.0,
    kl_weight=0.0,
    eps=1e-8
):
    """
    Masked reconstruction + KL divergence loss
    """

    # Masked reconstruction loss (MSE over observed entries only)
    se = (recon_x - x) ** 2
    masked_se = se * mask
    recon_loss = masked_se.sum(dim=1) / (mask.sum(dim=1) + eps)
    recon_loss = recon_loss.mean()

    # KL divergence
    kl_loss = -0.5 * torch.mean(
        torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1)
    )

    total_loss = recon_weight * recon_loss + kl_weight * kl_loss
    return total_loss, recon_loss.item(), kl_loss.item()


# =========================
# Training Function
# =========================

def train_masked_vae(
    model,
    dataloader,
    device,
    epochs=50,
    lr=1e-3
):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in range(1, epochs + 1):
        total_loss = 0.0
        total_rec = 0.0
        total_kl = 0.0

        for x, mask in dataloader:
            x = x.to(device).float()
            mask = mask.to(device).float()

            optimizer.zero_grad()
            recon, mu, logvar = model(x, mask)
            loss, rec, kl = masked_vae_loss(recon, x, mask, mu, logvar)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_rec += rec
            total_kl += kl

        print(
            f"Epoch {epoch:03d} | "
            f"Loss: {total_loss:.4f} | "
            f"Recon: {total_rec:.4f} | "
        )

    return model


# =========================
# Latent Feature Extraction
# =========================

def extract_latent_features(model, X, mask, device):
    """
    Returns latent feature vectors u_i = mu_i
    """
    model.eval()
    with torch.no_grad():
        X = X.to(device).float()
        mask = mask.to(device).float()
        mu, _ = model.encode(X, mask)
    return mu.cpu()

In [6]:
# ============================================
# 1. Load Iris Dataset
# ============================================
from sklearn.datasets import load_iris
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df["target"] = iris.target

In [97]:
# ============================================
# 2. Inject Missing Values (MCAR)
# ============================================

np.random.seed(42)

n_samples = df.shape[0]
missing_ratio = 0.0
n_missing_rows = int(missing_ratio * n_samples)

missing_rows = np.random.choice(df.index, size=n_missing_rows, replace=False)

for row in missing_rows:
    n_cols_missing = np.random.randint(1, len(iris.feature_names))
    cols_missing = np.random.choice(
        iris.feature_names, size=n_cols_missing, replace=False
    )
    df.loc[row, cols_missing] = np.nan

In [98]:
# ============================================
# 3. Create Feature Matrix and Mask
# ============================================

X = df[iris.feature_names].values.astype(np.float32)

# Binary mask: 1 = observed, 0 = missing
mask = (~np.isnan(X)).astype(np.float32)

# Fill missing values with zero (mask-aware)
X_filled = np.nan_to_num(X, nan=0.0)

In [99]:
# ============================================
# 4. Mask-aware Standardization
# ============================================
class TorchScaler:
    def fit(self, X: torch.Tensor):
        self.mean = X.mean(dim=0, keepdim=True)
        self.std = X.std(dim=0, unbiased=False, keepdim=True)
        self.std[self.std == 0] = 1.0
        return self
    def transform(self, X: torch.Tensor):
        return (X - self.mean) / self.std
    def inverse_transform(self, X: torch.Tensor):
        return X * self.std + self.mean

X_scaled = X_filled.copy()
X_tensor = torch.from_numpy(np.array(X_scaled))
scaler = TorchScaler().fit(X_tensor)
X_scaled = scaler.transform(X_tensor)
mask_tensor = torch.tensor(mask, dtype=torch.float32)

In [100]:

# ============================================
# 5. PyTorch Dataset and DataLoader
# ============================================
    
class MaskedDataset:
    def __init__(self, X, mask):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.mask = torch.tensor(mask, dtype=torch.float32)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.mask[idx]


dataset = MaskedDataset(X_scaled, mask)
torch.manual_seed(42)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

/tmp/ipykernel_3478032/1857059863.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32)


In [101]:
# =========================
# Example Usage (Skeleton)
# =========================
"""
Assumes your DataLoader yields (x, mask)
x    : tensor of shape [batch_size, input_dim]
mask : tensor of shape [batch_size, input_dim], binary {0,1}
"""

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = MaskedVAE(input_dim=INPUT_DIM, latent_dim=LATENT_DIM)
# model = train_masked_vae(model, dataloader, device)
# U = extract_latent_features(model, X_full, mask_full, device)

'\nAssumes your DataLoader yields (x, mask)\nx    : tensor of shape [batch_size, input_dim]\nmask : tensor of shape [batch_size, input_dim], binary {0,1}\n'

In [102]:
import time

In [103]:
start_time = time.time()

vae = MaskedVAE(input_dim=4, hidden_dim=3, latent_dim=2)
vae = train_masked_vae(vae, dataloader, device,epochs=1000,lr=1e-3)

end_time = time.time()

Epoch 001 | Loss: 4.0612 | Recon: 4.0612 | 
Epoch 002 | Loss: 4.0324 | Recon: 4.0324 | 
Epoch 003 | Loss: 3.9371 | Recon: 3.9371 | 
Epoch 004 | Loss: 3.9353 | Recon: 3.9353 | 
Epoch 005 | Loss: 3.9517 | Recon: 3.9517 | 
Epoch 006 | Loss: 3.9083 | Recon: 3.9083 | 
Epoch 007 | Loss: 3.8933 | Recon: 3.8933 | 
Epoch 008 | Loss: 3.9175 | Recon: 3.9175 | 
Epoch 009 | Loss: 3.8738 | Recon: 3.8738 | 
Epoch 010 | Loss: 3.8316 | Recon: 3.8316 | 
Epoch 011 | Loss: 3.8079 | Recon: 3.8079 | 
Epoch 012 | Loss: 3.8673 | Recon: 3.8673 | 
Epoch 013 | Loss: 3.7869 | Recon: 3.7869 | 
Epoch 014 | Loss: 3.7441 | Recon: 3.7441 | 
Epoch 015 | Loss: 3.7828 | Recon: 3.7828 | 
Epoch 016 | Loss: 3.6956 | Recon: 3.6956 | 
Epoch 017 | Loss: 3.6474 | Recon: 3.6474 | 
Epoch 018 | Loss: 3.7462 | Recon: 3.7462 | 
Epoch 019 | Loss: 3.6699 | Recon: 3.6699 | 
Epoch 020 | Loss: 3.7009 | Recon: 3.7009 | 
Epoch 021 | Loss: 3.6154 | Recon: 3.6154 | 
Epoch 022 | Loss: 3.6207 | Recon: 3.6207 | 
Epoch 023 | Loss: 3.5329 | Recon

In [104]:
elapsed = end_time - start_time
print(f"Code ran for {elapsed:.2f} seconds")

Code ran for 8.24 seconds


In [105]:
latent = extract_latent_features(vae, X_scaled,mask_tensor,device)
latent_np = latent.numpy()

In [106]:
latent_np

array([[  0.9683517 ,  -0.4030925 ],
       [  0.97714555,  -0.41217536],
       [  0.97581625,  -0.41081858],
       [  0.96903527,  -0.40383023],
       [  0.96565276,  -0.40031576],
       [  0.88982594,  -0.32203907],
       [  0.9612055 ,  -0.39575344],
       [  0.9656391 ,  -0.4003009 ],
       [  0.9758154 ,  -0.41083705],
       [  0.10375303,   0.36630014],
       [  0.9622431 ,  -0.39677155],
       [  0.9602276 ,  -0.39473245],
       [  0.41262746,   0.11898038],
       [  0.99113774,  -0.42665094],
       [  0.9710523 ,  -0.4058323 ],
       [  0.84171903,  -0.2723668 ],
       [  0.9546932 ,  -0.3889818 ],
       [  0.96253574,  -0.39709187],
       [ -9.255842  ,   9.870984  ],
       [  0.05904806,   0.4143396 ],
       [  0.9601884 ,  -0.39465097],
       [  0.9485966 ,  -0.38271266],
       [  0.9792404 ,  -0.41435903],
       [  0.9427534 ,  -0.37668237],
       [  0.94802225,  -0.38214236],
       [  0.9696803 ,  -0.40446794],
       [  0.94993883,  -0.3841029 ],
 

In [0]:
#stage 2

In [107]:
import numpy as np

# ============================================================
# Stage 2: Distributional Evidential Clustering (Option 3)
# ------------------------------------------------------------
# Input  : U  -> latent features from Stage 1 (masked AE/VAE)
# Output : fuzzy memberships, hard labels, cluster parameters
#
# Key design choices (intentional):
# - Clusters are Gaussian distributions in latent space
# - Memberships are evidential (fuzzy), not probabilistic
# - Distance is negative log-likelihood (not Euclidean)
# - Explicit cluster–cluster divergence regularization
# - Diagonal covariance for stability and efficiency
# - No EM, no mixture weights, no DST
# ============================================================


# ------------------------------------------------------------
# Negative log-likelihood under a diagonal Gaussian
# This replaces point-wise Euclidean distance
# ------------------------------------------------------------
def neg_log_likelihood(u, mu, var, eps=1e-6):
    """
    u   : latent feature vector of a sample (d,)
    mu  : cluster mean (d,)
    var : diagonal variance of cluster (d,)
    """
    d = u.shape[0]
    return 0.5 * (
        np.sum(np.log(var + eps)) +
        np.sum((u - mu) ** 2 / (var + eps)) +
        d * np.log(2 * np.pi)
    )


# ------------------------------------------------------------
# KL divergence between two diagonal Gaussian clusters
# Used to explicitly penalize cluster similarity
# ------------------------------------------------------------
def kl_divergence(mu1, var1, mu2, var2, eps=1e-6):
    """
    Computes KL( N(mu1,var1) || N(mu2,var2) )
    """
    return 0.5 * np.sum(
        np.log((var2 + eps) / (var1 + eps)) +
        (var1 + (mu1 - mu2) ** 2) / (var2 + eps) - 1.0
    )


# ------------------------------------------------------------
# Initialization
# - Means initialized from random latent points
# - Variances initialized to ones
# - Memberships initialized uniformly (maximum uncertainty)
# ------------------------------------------------------------
def initialize_clusters(U, n_clusters, seed=42):
    np.random.seed(seed)
    n, d = U.shape

    idx = np.random.choice(n, n_clusters, replace=False)
    mu = U[idx].copy()
    var = np.ones((n_clusters, d))           # diagonal covariance
    M = np.ones((n, n_clusters)) / n_clusters

    return mu, var, M


# ------------------------------------------------------------
# Compute divergence penalty for each cluster
# Δ_j = sum_{k≠j} KL(C_j || C_k)
# ------------------------------------------------------------
def compute_cluster_penalties(mu, var):
    c = mu.shape[0]
    Delta = np.zeros(c)

    for j in range(c):
        for k in range(c):
            if j != k:
                Delta[j] += kl_divergence(
                    mu[j], var[j],
                    mu[k], var[k]
                )
    return Delta


# ------------------------------------------------------------
# Membership update (Option 3)
# Implements Eq. (4) in the paper
# Uses direct exponentiation (no logsumexp)
# ------------------------------------------------------------
def update_memberships(U, mu, var, beta, gamma, eps=1e-12):
    """
    Memberships are updated using likelihood + divergence cost.
    This is NOT a posterior probability (unlike GMM).
    """
    n, _ = U.shape
    c = mu.shape[0]

    Delta = compute_cluster_penalties(mu, var)
    M = np.zeros((n, c))

    for i in range(n):
        numerators = np.zeros(c)

        for j in range(c):
            # Combined cost:
            # data-to-cluster fit + cluster-separation penalty
            cost_ij = (
                neg_log_likelihood(U[i], mu[j], var[j])
                + gamma * Delta[j]
            )

            numerators[j] = np.exp(-cost_ij / (beta - 1.0))

        # Normalize memberships to sum to 1
        M[i, :] = numerators / (np.sum(numerators) + eps)

    return M


# ------------------------------------------------------------
# Cluster parameter update
# Weighted maximum likelihood using evidential memberships
# Implements Eq. (5) and Eq. (6)
# ------------------------------------------------------------
def update_clusters(U, M, beta, eps=1e-8):
    n, d = U.shape
    c = M.shape[1]

    mu_new = np.zeros((c, d))
    var_new = np.zeros((c, d))
    M_beta = M ** beta

    for j in range(c):
        w = M_beta[:, j][:, None]
        denom = np.sum(w) + eps

        # Mean update
        mu_new[j] = np.sum(w * U, axis=0) / denom

        # Diagonal covariance update
        var_new[j] = np.sum(
            w * (U - mu_new[j]) ** 2, axis=0
        ) / denom

    return mu_new, var_new


# ------------------------------------------------------------
# Main optimization loop (Algorithm 1 in paper)
# ------------------------------------------------------------
def distributional_evidential_clustering(
    U,
    n_clusters,
    beta=2.0,
    gamma=0.05,
    max_iter=3,
    tol=1e-4,
    seed=42
):
    """
    Performs distributional evidential clustering on latent features.
    """
    mu, var, M = initialize_clusters(U, n_clusters, seed)

    for it in range(max_iter):
        mu_old = mu.copy()

        # Step 1: update memberships
        M = update_memberships(U, mu, var, beta, gamma)

        # Step 2: update cluster distributions
        mu, var = update_clusters(U, M, beta)

        # Convergence check on cluster means
        shift = np.max(np.linalg.norm(mu - mu_old, axis=1))
        if shift < tol:
            print(f"Converged at iteration {it}")
            break

    labels = np.argmax(M, axis=1)

    return {
        "memberships": M,
        "labels": labels,
        "mu": mu,
        "var": var
    }


In [108]:
latent_np.shape

(150, 2)

In [124]:

result = distributional_evidential_clustering(
    latent_np,
    n_clusters=3,
    beta=2,
    gamma=0.00000001,
    max_iter=50,
    tol=1e-8,
    seed=42
)

Converged at iteration 45


In [125]:
result

{'memberships': array([[8.40629411e-08, 9.99999916e-01, 8.47968548e-36],
        [2.11448287e-07, 9.99999789e-01, 2.02307102e-35],
        [1.70955735e-07, 9.99999829e-01, 1.64870321e-35],
        [8.67038691e-08, 9.99999913e-01, 8.70940088e-36],
        [7.99019499e-08, 9.99999920e-01, 8.19155369e-36],
        [9.99999990e-01, 2.28785578e-12, 1.61599228e-28],
        [9.31160714e-08, 9.99999907e-01, 9.80404276e-36],
        [7.99035596e-08, 9.99999920e-01, 8.19240954e-36],
        [1.71176344e-07, 9.99999829e-01, 1.65074554e-35],
        [9.99999995e-01, 0.00000000e+00, 1.17395208e-26],
        [8.76187027e-08, 9.99999912e-01, 9.16931043e-36],
        [1.00251127e-07, 9.99999900e-01, 1.06178501e-35],
        [9.99999993e-01, 0.00000000e+00, 2.36542665e-27],
        [1.00383297e-05, 9.99989962e-01, 8.82827550e-34],
        [9.84632027e-08, 9.99999902e-01, 9.77371159e-36],
        [9.99999990e-01, 1.20907798e-43, 2.15535028e-28],
        [1.99582155e-07, 9.99999800e-01, 2.18553987e-35],

In [126]:
labels = result["labels"]
memberships = result["memberships"]
df["pred"]=labels

In [127]:
from sklearn.metrics import adjusted_rand_score


ari = adjusted_rand_score(df["target"], df["pred"])
print("Adjusted Rand Index:", 100*ari)

Adjusted Rand Index: 57.62031253929242


In [93]:
import numpy as np

U = latent_np
n_clusters = 3

mu, var, M = initialize_clusters(U, n_clusters)

print("mu shape:", mu.shape)   # (4, 3)
print("var shape:", var.shape) # (4, 3)
print("M shape:", M.shape)     # (10, 4)

mu shape: (3, 2)
var shape: (3, 2)
M shape: (150, 3)


In [94]:
mu

array([[-2.0261173,  1.4990529],
       [-3.9042404,  1.3606722],
       [ 1.9128268,  1.6425785]], dtype=float32)

In [96]:
var

array([[1., 1.],
       [1., 1.],
       [1., 1.]])